In [1]:
pip install pymysql


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pymysql pandas SQLAlchemy


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pymysql
import pandas as pd

# Replace with your RDS credentials
host = "stratus-db-reporting.cszd17ujdumz.us-east-1.rds.amazonaws.com"
user = "user_kleman"
password = "bCFI97LDj9UP2y!"
database = "analytics_db"

query = r"""
WITH tables_with_account AS (
    SELECT 
        TABLE_NAME
    FROM information_schema.COLUMNS
    WHERE TABLE_SCHEMA = DATABASE()
      AND COLUMN_NAME = 'Account'
      # AND COLUMN_NAME like '%dpd%'
), loan_tape AS (
    SELECT *
    FROM analytics_db.loan_tape
),
id_mapping AS (
    SELECT DISTINCT Account, `Salesforce_Loan Applicant ID`, `Reference (XERO)`, `Borrower Name` 
    FROM analytics_db.applicant_id_mapping
), 
flight_training AS (
    SELECT Account, 
        Borrower as BorrowerID, 
        Name, 
        `Date of Loan Application` as Loan_Application_Date,
        Citizenship,
        Country,
        Birthday, 
        School,
        NPS, 
        `Flight School State` as Flight_School_State,
        `Flight School Tier` as Flight_School_Tier,
        Program,
        Particulars,
        `Loan Amount` as Loan_Amount,
        `Total Hours` as Total_Hours,
        `0 hours` as `0_hours`,
        `40/50 hours` as `40_50_hours`,
        `70/120 hours` as `70_120_hours`,
        `170/180 hours` as `170_180_hours`,
        `200/220 hours` as `200_220_hours`,
        `230 hours` as `230_hours`,
        `No. of Months`,
        `Funding Status`,
        `Date of Last Funding`,
        `Medical Class`,
        PPL,
        IFR, 
        `COMM SE`,
        `COMM Multi`,
        CFI, 
        CFII,
        MEI,
        ATP,
        `Certificates based on FAA`,
        `Other Certificates based on FAA`,
        Employment
    FROM analytics_db.raw_flight_training
),
payment_historical AS (
    SELECT Account, YearMonth, LoanAccount, DateDue, 
           DateRec, LoanBalance, TotalAmount, ToPrincipal_RegPmt, 
           ToInterest_RegPmt, ToPRincipal_Paydown, ToPrincipal_Payoff,
           Notes, DaysPastDue
    FROM analytics_db.payment_with_dpd
),
net_disbursement AS (
    -- revisit later if disbursement is needed
    SELECT *
    FROM analytics_db.net_disbursements
),
scheduled_payment AS (
    SELECT *
         , row_number() over (partition by base.Account, base.Payment_Schedule_ID order by base.Account, base.Date) as rnPS
    FROM (
        SELECT DISTINCT a.Account, a.Date, `Payment Schedule ID` AS Payment_Schedule_ID
             , `Expected Payment Amount` as Expected_Payment_Amount
             , `Interest Payment` AS Interest_Payment
             , `Principal Payment` AS Principal_Payment
             , ROUND(`Interest Payment` + `Principal Payment`, 2) as Total
             , SUBSTRING(a.Date, 1, 7) as YearMonth
        FROM analytics_db.scheduled_payment_tape a
        INNER JOIN loan_tape b on a.Account = b.Account
    ) base 
    WHERE 1 = 1
      -- AND base.Account in (100000901, 100000201, 100000301, 100000401)
      AND Expected_Payment_Amount = Total
),
scheduled_payment2 AS (
    SELECT *
         , FLOOR((rn - 1) / 12) as ScheduleYearNbr
         , ROUND(Interest_Payment * 100 / Expected_Payment_Amount, 2) as InterestRate
    FROM (
        SELECT *
             , row_number() over (partition by Account order by Account, Date) as rn
        FROM scheduled_payment
        WHERE rnPS = 1    
    ) base 
),
payment_history AS (
    SELECT Account
         , DateRec
         , YearMonth
         , AVG(ExpectedPaymentAmount) as ExpectedPaymentAmount
         , SUM(ActualPaymentAmount) as ActualPaymentAmount
    FROM (
        SELECT DISTINCT Account,
               ExpectedPaymentAmount,
               ROUND(ToInterest_RegPmt + ToPrincipal_RegPmt, 2) AS ActualPaymentAmount,
               CAST(DateRec as Date) as DateRec,
               SUBSTRING(DateRec, 1, 7) as YearMonth
        FROM analytics_db.payment_dpd_raw
    ) base
    GROUP BY Account, DateRec, YearMonth
)
, payment_gap_raw AS (
    SELECT c.Account
         , c.Date as ScheduledDate
         , c.YearMonth as ScheduledYM
         , c.ScheduleYearNbr
         , b.Date as NextScheduledDate
         , c.Payment_Schedule_ID
         , c.Expected_Payment_Amount
         , a.ActualPaymentAmount
         , CASE WHEN a.DateRec is NULL THEN c.Date ELSE a.DateRec END as PaymentDate
         , a.YearMonth as PaymentYM
         , DATEDIFF(DateRec, c.Date) as DPD
         , DATEDIFF(
               CASE 
                 WHEN a.DateRec is NULL 
                  OR a.ActualPaymentAmount is NULL 
                  OR a.ActualPaymentAmount < 0
                  OR c.Expected_Payment_Amount > a.ActualPaymentAmount THEN b.Date 
                 ELSE a.DateRec 
               END,
               c.Date
           ) as DPD2
         , CASE 
             WHEN a.ActualPaymentAmount is NOT NULL 
                  AND c.Expected_Payment_Amount > a.ActualPaymentAmount THEN "missed payment" 
             WHEN a.ActualPaymentAmount is NULL THEN "missed payment" 
             ELSE "paid" 
           END as PaidInd
    FROM scheduled_payment2 c 
    LEFT JOIN scheduled_payment2 b 
           ON c.Account = b.Account 
          AND c.rn = b.rn - 1
    LEFT JOIN payment_history a 
           ON a.Account = c.Account 
          AND a.YearMonth = c.YearMonth 
    WHERE c.YearMonth <= "2025-12"
    ORDER BY c.Account, c.YearMonth
),
payment_gap_raw2 AS ( 
    SELECT
        q.*,
        -- Group segments between zeroes
        SUM(CASE WHEN DPD2 = 0 THEN 1 ELSE 0 END) OVER (
            PARTITION BY Account
            ORDER BY Payment_Schedule_ID
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS zero_grp
    FROM payment_gap_raw q
), 
payment_gap_raw2_result AS (
    SELECT
        a.*,
        CASE
            WHEN DPD2 > 0 THEN
                SUM(DPD2) OVER (
                    PARTITION BY Account, zero_grp, PaidInd
                    ORDER BY Payment_Schedule_ID
                    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                )
            ELSE 0
        END AS DPD2_running_sum_all,
        CASE
            WHEN DPD2 > 0 THEN
                SUM(DPD2) OVER (
                    PARTITION BY Account, zero_grp, PaidInd, ScheduleYearNbr
                    ORDER BY Payment_Schedule_ID
                    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                )
            ELSE 0
        END AS DPD2_running_sum_yrly
    FROM payment_gap_raw2 as a
    ORDER BY Account, Payment_Schedule_ID
)
, gaps_in_payment AS (
    SELECT *
         , CASE WHEN DPD_yrly > 60 THEN 1 ELSE 0 END as DPD60Ind_yrly
         , CASE WHEN DPD_yrly > 90 THEN 1 ELSE 0 END as DPD90Ind_yrly
         , CASE WHEN DPD_yrly > 120 THEN 1 ELSE 0 END as DPD120Ind_yrly
         , CASE WHEN DPD_alltime > 60 THEN 1 ELSE 0 END as DPD60Ind_alltime
         , CASE WHEN DPD_alltime > 90 THEN 1 ELSE 0 END as DPD90Ind_alltime
         , CASE WHEN DPD_alltime > 120 THEN 1 ELSE 0 END as DPD120Ind_alltime
    FROM (
        SELECT DISTINCT Account
             , ScheduleYearNbr
             , PaidInd
             , MIN(ScheduledYM) OVER (PARTITION BY Account, ScheduleYearNbr) as minYM_yrly
             , MAX(ScheduledYM) OVER (PARTITION BY Account, ScheduleYearNbr) as maxYM_yrly
             , MAX(DPD2_running_sum_yrly) OVER (PARTITION BY Account, ScheduleYearNbr) as DPD_yrly
             , MAX(DPD2_running_sum_all) OVER (PARTITION BY Account) as DPD_alltime
        FROM payment_gap_raw2_result
    ) base
    WHERE PaidInd = "missed payment"
)
, interest_rate_summary AS (
    SELECT Account
         , ScheduleYearNbr
         , MIN(YearMonth) as minYearMo
         , MAX(YearMonth) as maxYearMo
         , MIN(InterestRate) as minInterestRate
         , ROUND(AVG(InterestRate), 2) as avgInterestRate
         , MAX(InterestRate) as maxInterestRate
    FROM scheduled_payment2 
    GROUP BY Account, ScheduleYearNbr
)
, borrower_demographics_FT AS (
    SELECT Account
         , BorrowerID
         , Name
         , Citizenship
         , Country
         , Birthday
         , CEILING(DATEDIFF("2025-12-31", Birthday) / 365.25) as Age
         , Employment
    FROM flight_training 
    ORDER BY Account
)
, borrower_loan_information_FT AS (
    SELECT Account
         , Loan_Amount
         , `Funding Status` AS Funding_Status
         , `Date of Last Funding` AS Date_Last_Funding
    FROM flight_training 
    ORDER BY Account
)
, borrower_flight_records_FT AS (
    SELECT Account
         , School
         , NPS
         , Flight_School_State
         , Flight_School_Tier
         , TRIM(REGEXP_REPLACE(Flight_School_Tier, '\\s*\\(.*$', '')) AS Flight_School_Tier_Clean
         , Program
         , TRIM(REGEXP_REPLACE(Program, '\\s*\\(.*$', '')) AS Program_Clean
         , Particulars
         , Total_Hours
         , 0_hours
         , 40_50_hours
         , 70_120_hours
         , 170_180_hours
         , 200_220_hours
         , 230_hours
         , `No. of Months` AS Num_Months_Training_Completed
         , `Medical Class` AS Medical_Class
         , PPL
         , IFR
         , `COMM SE` AS COMM_SE
         , `COMM Multi` AS COMM_Multi
         , CFI
         , CFII
         , MEI
         , ATP
         , `Other Certificates based on FAA` AS OTHER_CERT_FAA
    FROM flight_training 
    ORDER BY Account
), 
latest_tmo_loan_information AS (
    SELECT *
    FROM (
        SELECT * 
             , row_number() over (partition by Account order by date_added desc) as rn
        FROM analytics_db.tmo_all_loans_consolidated 
    ) base
    WHERE rn = 1
), 
tmo_all_loans AS (
    SELECT * FROM analytics_db.tmo_all_loans
)

-- FINAL RESULT TO EXTRACT
-- SELECT * FROM loan_tape;
-- SELECT * FROM gaps_in_payment;
-- SELECT * FROM interest_rate_summary;
-- SELECT * FROM borrower_demographics_FT;
-- SELECT * FROM borrower_loan_information_FT;
-- SELECT * FROM borrower_flight_records_FT;
-- SELECT * FROM latest_tmo_loan_information;
-- SELECT * FROm scheduled_payment2;
SELECT * FROM tmo_all_loans
"""

try:
    conn = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database,
        port=3306,
        cursorclass=pymysql.cursors.DictCursor
    )

    print("Connected to AWS RDS successfully!")

    with conn.cursor() as cursor:
        cursor.execute(query)
        result = cursor.fetchall()
        # print(result)
    result_pd = pd.DataFrame(result)
    print(result_pd.head())

    # --- Save to CSV ---
    # output_path = "/Users/klemanroy/Downloads/stratus/v2/aa_loan_tape_extract.csv"
    # output_path = "/Users/klemanroy/Downloads/stratus/v2/aa_gaps_in_payment_extract.csv"
    # output_path = "/Users/klemanroy/Downloads/stratus/v2/aa_interest_rate_summary_extract.csv"
    # output_path = "/Users/klemanroy/Downloads/stratus/v2/aa_borrower_demographics_FT_extract.csv"
    # output_path = "/Users/klemanroy/Downloads/stratus/v2/aa_borrower_loan_information_FT_extract.csv"
    # output_path = "/Users/klemanroy/Downloads/stratus/v2/aa_borrower_flight_records_FT_extract.csv"
    # output_path = "/Users/klemanroy/Downloads/stratus/v2/aa_latest_tmo_loan_information_extract.csv"
    # output_path = "/Users/klemanroy/Downloads/stratus/v2/aa_scheduled_payment_extract.csv"
    output_path = "/Users/klemanroy/Downloads/stratus/v2/data_extract_from_sql/aa_latest_tmo_all_loan_extract.csv"
    result_pd.to_csv(output_path, index=False)
    
    print(f"Saved {len(result_pd)} rows to {output_path}")
    print(f"Saved {len(set(result_pd["Account"]))} unique Accounts")

except Exception as e:
    print("Connection error:", e)


Connected to AWS RDS successfully!
   ach  hold    account          borrower_name     by_last_name first_name  \
0    0     0  100000101  JACOB WILLIAM SHAWHAN   SHAWHAN, JACOB      JACOB   
1    0     0  100000201         MICHAEL KNIGHT  KNIGHT, MICHAEL    MICHAEL   
2    1     0  100000301  PATRICK KELLY SHEA JR    SHEA, PATRICK    PATRICK   
3    1     0  100000401         EDWARD RAMIREZ  RAMIREZ, EDWARD     EDWARD   
4    1     0  100000501          NIKITA BANDYK   BANDYK, NIKITA     NIKITA   

     mi last_name interest_paid_to payment_due_date  ...  \
0  None   SHAWHAN       2023-08-15       2023-09-15  ...   
1  None    KNIGHT       2023-05-15       2023-06-15  ...   
2  None      SHEA       2024-05-15       2024-06-15  ...   
3  None   RAMIREZ       2024-05-15       2024-06-15  ...   
4  None    BANDYK       2024-05-15       2024-06-15  ...   

  forbearance_3_start_date  forbearance_3_amount  forbearance_3_end_date  \
0                     None                   NaN           

In [2]:
print(f"Saved {len(set(result_pd["Account"]))} unique Accounts")

NameError: name 'result_pd' is not defined